# Installing required libraries for Unsloth, Xformers, and other dependencies.


In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install transformers datasets

# Mounting the drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the FastLanguageModel from the Unsloth library with 4-bit precision.


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# Configuring the FastLanguageModel with PEFT (Parameter-Efficient Fine-Tuning) for optimized model training.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


# Loading a custom recipe dataset, formatting it for model input/output, and converting it into a Hugging Face dataset.


In [5]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("/content/Recipe_Dataset_Final.csv")

df['input'] = df.apply(lambda row: f"Dietary tags: {row['tag']}. Available ingredients: {row['ingredients']}.", axis=1)

df['output'] = df.apply(lambda row: (
    f"Recipe Name: {row['title']}.\n"
    f"Recipe Directions: {row['directions']}.\n"
    f"Nutrition - Calories: {row['calories']} kcal, Protein: {row['protein']} g, Fat: {row['fat']} g, Sodium: {row['sodium']} mg."
), axis=1)

dataset = Dataset.from_pandas(df)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the dietary tags and available ingredients, generate a detailed recipe that includes the following:
1. Recipe Name
2. Recipe Directions
3. Nutritional Values (Calories, Protein, Fat, Sodium)

Ensure that the recipe fits the dietary restrictions specified by the tags and make adjustments to the ingredients if necessary to meet those requirements.

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/5662 [00:00<?, ? examples/s]

# Setting up the SFTTrainer from `trl` for fine-tuning the model with float16 or bfloat16, based on hardware support.


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

fp16_enabled = not is_bfloat16_supported()
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        max_steps=150,
        learning_rate=2e-4,
        output_dir="outputs",
        fp16=fp16_enabled,
        bf16=not fp16_enabled,
    ),
)

trainer.train()

Map:   0%|          | 0/5662 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,662 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 150
 "-____-"     Number of trainable parameters = 57,016,320


Step,Training Loss


TrainOutput(global_step=150, training_loss=0.6655878194173177, metrics={'train_runtime': 3480.9334, 'train_samples_per_second': 0.345, 'train_steps_per_second': 0.043, 'total_flos': 4.383665720856576e+16, 'train_loss': 0.6655878194173177, 'epoch': 0.21193924408336276})

# Saving the trained model in drive

In [10]:
save_path = "/content/drive/MyDrive/Nebula9.ai"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/MyDrive/Nebula9.ai/tokenizer_config.json',
 '/content/drive/MyDrive/Nebula9.ai/special_tokens_map.json',
 '/content/drive/MyDrive/Nebula9.ai/tokenizer.json')